In [1]:
import requests

def chat_with_ollama(prompt, model="qwen2.5:3b-instruct"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  # disable streaming
    }
    response = requests.post(url, json=payload)
    data = response.json()
    return data["response"]

if __name__ == "__main__":
    user_input = "Explain me string theory in simple terms."
    reply = chat_with_ollama(user_input)
    print(f"Ollama: {reply}")

Ollama: String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity, which are two of the most fundamental theories in physics. In this theory, at the smallest scales of nature, instead of particles (like electrons or quarks), everything is composed of vibrating, tiny, one-dimensional objects called strings.

Imagine if you could stretch a very thin thread very long to create something like a guitar string. Each note that a guitar produces has different frequencies and harmonics because the string vibrates at different rates. In the same way, strings in string theory can vibrate at various frequencies or "modes," each corresponding to a different particle.

There are several types of particles (bosons) that interact with one another through forces (gauge bosons), such as gravity. These interactions are explained by how these vibrating strings interact and vibrate differently. For example, the vibrations associated with a parti

In [ ]:
import requests
import json

def chat_with_ollama(prompt, model="llama3.2:latest"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": True  # enable streaming
    }

    with requests.post(url, json=payload, stream=True) as resp:
        for line in resp.iter_lines():
            if line:  # skip empty lines
                data = json.loads(line.decode("utf-8"))
                if "response" in data:
                    yield data["response"]
                if data.get("done", False):
                    break

if __name__ == "__main__":
    user_input = "Tell me about Golden Cross in trading."
    print("Ollama:", end=" ", flush=True)
    for chunk in chat_with_ollama(user_input):
        print(chunk, end="", flush=True)
    print()

Ollama: The Golden Cross is a technical analysis indicator used to identify the potential start of a new uptrend in a stock's price. It occurs when the 50-day moving average (MA) crosses above the 200-day MA, indicating that the shorter-term trend has shifted into the long-term trend.

Here are some key aspects of the Golden Cross:

1. **Signal generation**: The Golden Cross is generated when the 50-day MA crosses above the 200-day MA.
2. **Confirmation**: A buy signal is confirmed when the stock price closes above its 50-day MA after the cross-over.
3. **Mean reversion**: Some traders believe that a Golden Cross can also be a sign of mean reversion, as it indicates that the short-term trend may reverse and become more aligned with the long-term trend.
4. **Uptrend confirmation**: The Golden Cross is considered a bullish indicator, confirming an uptrend and indicating that the stock's price is likely to continue rising.

**When to use:**

1. **Buy signals**: Use the Golden Cross as a b

In [ ]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# 1. Define your LLM (Ollama backend)
llm = Ollama(model="llama3.2:latest", base_url="http://localhost:11434")  
# If running in K8s with ingress: base_url="http://your-ollama-service"

# 2. Chain 1: Generate outline
prompt1 = PromptTemplate(
    input_variables=["subject"],
    template="Create a 3-point outline for a blog on {subject}."
)
chain1 = LLMChain(llm=llm, prompt=prompt1)

# 3. Chain 2: Expand outline into a blog
prompt2 = PromptTemplate(
    input_variables=["outline"],
    template="Write a blog article based on this outline:\n{outline}"
)
chain2 = LLMChain(llm=llm, prompt=prompt2)

# 4. Combine into sequential chain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# 5. Run the chain
print(overall_chain.invoke("Kubernetes autoscaling"))

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_32212/2714210623.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2:latest", base_url="http://localhost:11434")
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_32212/2714210623.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain1 = LLMChain(llm=llm, prompt=prompt1)




> Entering new SimpleSequentialChain chain...
Here is a 3-point outline for a blog on Kubernetes autoscaling:

I. Introduction to Kubernetes Autoscaling

* Brief overview of Kubernetes and its benefits
* Explanation of what Kubernetes autoscaling is and its importance in modern cloud-native applications
* Discussion of the challenges and limitations of traditional scaling approaches (e.g. static scaling, dynamic scaling)

II. Types of Kubernetes Autoscaling Strategies

* Horizontal Pod Autoscaling (HPA): a popular approach to automatically scale the number of replicas based on CPU utilization or other metrics
* Vertical Pod Autoscaling: a strategy for dynamically adjusting resources allocated to individual pods based on their workload needs
* Batch-based Autoscaling: an approach that scales applications in batches, often used for applications with predictable and recurring workloads

III. Best Practices for Implementing Kubernetes Autoscaling

* Overview of key considerations when im

In [9]:
# Trading Strategy: Volume Surge + 30-Day Low Reversal
import yfinance as yf
import pandas as pd
import datetime

# === Config ===
TICKERS = ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GNFC.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GPPL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JUSTDIAL.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KANSAINER.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KNRCON.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MASTEK.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PEL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RAYMOND.NS', 'RAYMONDLSL.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RENUKA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'ROUTE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RTNINDIA.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SWSOLAR.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TANLA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS']
LOOKBACK = 30
START = (datetime.date.today() - datetime.timedelta(days=60)).strftime("%Y-%m-%d")
END = datetime.date.today().strftime("%Y-%m-%d")

results = []

for ticker in TICKERS:
    try:
        df = yf.download(ticker, start=START, end=END, progress=False, multi_level_index=False)

        if len(df) < LOOKBACK + 2:
            continue  # Not enough data

        df["30d_low"] = df["Low"].rolling(LOOKBACK).min()
        df["30d_high"] = df["High"].rolling(LOOKBACK).max()

        # Take last 2 rows (Yesterday, Today)
        today = df.iloc[-1]
        yesterday = df.iloc[-2]

        cond1 = (yesterday["Low"] == df["Low"].iloc[-LOOKBACK-1:-1].min())  # Yesterday's low = lowest of last 30
        cond2 = (today["Volume"] > 1.8 * yesterday["Volume"])              # Volume surge
        cond3 = (today["Low"] > yesterday["Low"])                         # Higher low
        cond4 = (today["Open"] > 20)                                      # Price filter
        cond5 = (today["High"] < df["High"].iloc[-LOOKBACK:].max())       # Not 30d breakout

        if cond1 and cond2 and cond3 and cond4 and cond5:
            results.append({
                "Ticker": ticker,
                "Yesterday_Low": yesterday["Low"],
                "Today_Low": today["Low"],
                "Today_High": today["High"],
                "Today_Volume": today["Volume"],
                "Yesterday_Volume": yesterday["Volume"]
            })
    except Exception as e:
        print(f"Error with {ticker}: {e}")

# Final Output
if results:
    res_df = pd.DataFrame(results)
    print("Stocks satisfying strategy:\n")
    display(res_df)
else:
    print("No stocks satisfy the strategy today.")


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_24041/445963484.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_24041/445963484.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_24041/445963484.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_24041/445963484.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False, multi_level_index=False)


Stocks satisfying strategy:



,Ticker,Yesterday_Low,Today_Low,Today_High,Today_Volume,Yesterday_Volume
0,NUVAMA.NS,6155.0,6159.0,6520.0,355864.0,184611.0


In [25]:
# Step 1: Import necessary libraries
import yfinance as yf
import pandas as pd
import talib as ta
from datetime import datetime
from IPython.display import display

print("Libraries imported successfully.")

# ==============================================================================
# Step 2: All Strategy Parameters
# ==============================================================================

# -- Ticker and Date Settings --
TICKERS_TO_SCAN = [
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS'
]
MARKET_INDEX = '^NSEI' # Use '^NSEI' for Nifty 50, '^GSPC' for S&P 500
START_DATE = "2010-01-01"
END_DATE = datetime.now().strftime('%Y-%-m-%d')

# -- Indicator Settings --
ST_ATR_PERIOD = 10
ST_MULTIPLIER = 3.0
VOL_MA_PERIOD = 20
MARKET_MA_PERIOD = 50

# -- Exit Strategy Settings --
SL_PERCENTAGE = 0.02  # Represents 2%

# ==============================================================================
# Step 3: Feature Toggles (Enable/Disable Rules Here)
# ==============================================================================
# Set a rule to 'True' to enable it, or 'False' to disable it.

ENABLE_SUPERTREND_FLIP = False    # Core Signal: Confirms a trend change just happened.
ENABLE_R1_S1_BREAK = True        # Confirmation: Price broke a key pivot level.
ENABLE_MOMENTUM_CANDLE = False    # Confirmation: The candle shows strong momentum (close > open).
ENABLE_VOLUME_CONFIRMATION = False# Confirmation: Volume is above average.
ENABLE_MARKET_CONTEXT = False     # Filter: Only trade in the direction of the broader market.

# ==============================================================================
# Step 4: Helper Function to Calculate Supertrend (No Changes Here)
# ==============================================================================
def calculate_supertrend(df, atr_period, multiplier):
    high, low, close = df['High'], df['Low'], df['Close']
    atr = ta.ATR(high, low, close, timeperiod=atr_period)
    
    supertrend_line, final_upper, final_lower = ([0.0] * len(df) for _ in range(3))
    
    for i in range(1, len(df)):
        upper_band = ((high.iloc[i] + low.iloc[i]) / 2) + (multiplier * atr.iloc[i])
        lower_band = ((high.iloc[i] + low.iloc[i]) / 2) - (multiplier * atr.iloc[i])

        final_upper[i] = upper_band if upper_band < final_upper[i-1] or close.iloc[i-1] > final_upper[i-1] else final_upper[i-1]
        final_lower[i] = lower_band if lower_band > final_lower[i-1] or close.iloc[i-1] < final_lower[i-1] else final_lower[i-1]
            
        if supertrend_line[i-1] == final_upper[i-1] and close.iloc[i] <= final_upper[i]:
            supertrend_line[i] = final_upper[i]
        elif supertrend_line[i-1] == final_upper[i-1] and close.iloc[i] > final_upper[i]:
            supertrend_line[i] = final_lower[i]
        elif supertrend_line[i-1] == final_lower[i-1] and close.iloc[i] >= final_lower[i]:
            supertrend_line[i] = final_lower[i]
        elif supertrend_line[i-1] == final_lower[i-1] and close.iloc[i] < final_lower[i]:
            supertrend_line[i] = final_upper[i]

    df['Supertrend_Line'] = supertrend_line
    df['Supertrend_Direction'] = (close > df['Supertrend_Line']).astype(int).replace(0, -1)
    return df

# ==============================================================================
# Step 5: Main Scanner Logic
# ==============================================================================
print("Starting trading strategy scanner...")

# --- Determine Broader Market Context ---
is_market_bullish = True # Default to true if context is disabled or fails
try:
    if ENABLE_MARKET_CONTEXT:
        index_df = yf.download(MARKET_INDEX, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)
        index_df['MA'] = index_df['Close'].rolling(window=MARKET_MA_PERIOD).mean()
        is_market_bullish = index_df.iloc[-1]['Close'] > index_df.iloc[-1]['MA']
        market_status = "Bullish" if is_market_bullish else "Bearish"
        print(f"Market Context ({MARKET_INDEX}): {market_status}")
    else:
        print("Market Context: Disabled")
except Exception:
    print(f"Could not determine market context. Defaulting to allow all trades.")

final_signals = []
print(f"Scanning {len(TICKERS_TO_SCAN)} tickers...")

for ticker in TICKERS_TO_SCAN:
    try:
        df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)
        if len(df) < MARKET_MA_PERIOD: continue

        # --- Calculate Indicators ---
        df = calculate_supertrend(df, ST_ATR_PERIOD, ST_MULTIPLIER)
        df['Volume_MA'] = df['Volume'].rolling(window=VOL_MA_PERIOD).mean()
        df['PP'] = (df['High'].shift(1) + df['Low'].shift(1) + df['Close'].shift(1)) / 3
        df['R1'] = (2 * df['PP']) - df['Low'].shift(1)
        df['R2'] = df['PP'] + (df['High'].shift(1) - df['Low'].shift(1))
        df['S1'] = (2 * df['PP']) - df['High'].shift(1)
        df['S2'] = df['PP'] - (df['High'].shift(1) - df['Low'].shift(1))
        df.dropna(inplace=True)
        if df.empty: continue
        
        latest = df.iloc[-1]
        prev = df.iloc[-2]

        # ==============================================================================
        # MODIFIED SECTION: Apply Entry Rules Based on Toggles
        # ==============================================================================
        signal = "No Signal"

        # --- Define Long Conditions ---
        long_cond_st_flip = prev['Supertrend_Direction'] == -1 and latest['Supertrend_Direction'] == 1
        long_cond_breakout = latest['Close'] > latest['R1']
        long_cond_candle = latest['Close'] > latest['Open']
        long_cond_volume = latest['Volume'] > latest['Volume_MA']
        long_cond_market = is_market_bullish

        if (not ENABLE_SUPERTREND_FLIP or long_cond_st_flip) and \
           (not ENABLE_R1_S1_BREAK or long_cond_breakout) and \
           (not ENABLE_MOMENTUM_CANDLE or long_cond_candle) and \
           (not ENABLE_VOLUME_CONFIRMATION or long_cond_volume) and \
           (not ENABLE_MARKET_CONTEXT or long_cond_market):
            signal = "Long"

        # --- Define Short Conditions ---
        short_cond_st_flip = prev['Supertrend_Direction'] == 1 and latest['Supertrend_Direction'] == -1
        short_cond_breakdown = latest['Close'] < latest['S1']
        short_cond_candle = latest['Close'] < latest['Open']
        short_cond_volume = latest['Volume'] > latest['Volume_MA']
        short_cond_market = not is_market_bullish
        
        # Only check for a short signal if a long one wasn't already found
        if signal == "No Signal":
            if (not ENABLE_SUPERTREND_FLIP or short_cond_st_flip) and \
               (not ENABLE_R1_S1_BREAK or short_cond_breakdown) and \
               (not ENABLE_MOMENTUM_CANDLE or short_cond_candle) and \
               (not ENABLE_VOLUME_CONFIRMATION or short_cond_volume) and \
               (not ENABLE_MARKET_CONTEXT or short_cond_market):
                signal = "Short"
        
        # --- Store Signal and Calculate Exits ---
        if signal != "No Signal":
            stop_loss = latest['Supertrend_Line'] * (1 - SL_PERCENTAGE) if signal == "Long" else latest['Supertrend_Line'] * (1 + SL_PERCENTAGE)
            profit_target = latest['R2'] if signal == "Long" else latest['S2']

            final_signals.append({
                'Date': latest.name.strftime('%Y-%-m-%d'), 'Ticker': ticker, 'Signal': signal,
                'Entry_Price': f"{latest['Close']:.2f}", 'Stop_Loss': f"{stop_loss:.2f}",
                'Profit_Target': f"{profit_target:.2f}"
            })
            print(f"--- Signal Found for {ticker}: {signal} ---")

    except Exception as e:
        print(f"Could not analyze {ticker}. Reason: {e}")

# ==============================================================================
# Step 6: Output Results
# ==============================================================================
print("\n--- Scan Complete ---")

if final_signals:
    signals_df = pd.DataFrame(final_signals)
    output_filename = f'trading_signals_{datetime.now().strftime("%Y-%m-%d")}.csv'
    signals_df.to_csv(output_filename, index=False)
    
    print(f"\nSuccessfully generated {len(signals_df)} signals.")
    print(f"Results saved to '{output_filename}'")
    display(signals_df)
else:
    print(f"\nNo trading signals found for the given tickers and enabled rules.")

Libraries imported successfully.
Starting trading strategy scanner...
Market Context: Disabled
Scanning 4 tickers...


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_26144/457545484.py:95: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_26144/457545484.py:95: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_26144/457545484.py:95: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)


--- Signal Found for HDFCBANK.NS: Long ---
--- Signal Found for INFY.NS: Long ---

--- Scan Complete ---

Successfully generated 2 signals.
Results saved to 'trading_signals_2025-09-19.csv'


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_26144/457545484.py:95: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)


,Date,Ticker,Signal,Entry_Price,Stop_Loss,Profit_Target
0,2025-9-18,HDFCBANK.NS,Long,976.90,0.00,978.47
1,2025-9-18,INFY.NS,Long,1540.60,0.00,1533.73


In [37]:
import yfinance as yf
infy = yf.Ticker("TCS.NS")
infy.mutualfund_holders
infy.news
infy.get_news(count=10, tab='all')

# import yfinance as yf
# response = yf.screen("aggressive_small_caps")
# print(response)  # Display the filtered stocks

[{'id': 'cd779a9a-0ebe-3d30-a502-6040f8474753',
  'content': {'id': 'cd779a9a-0ebe-3d30-a502-6040f8474753',
   'contentType': 'STORY',
   'title': 'Review & Preview: The Rally Stumbles',
   'description': '',
   'summary': 'Federal Reserve Chair Jerome Powell took the steam out of the market’s rally, with equities closing lower on Tuesday.  The S&P 500 was off 0.6%, the Dow Jones Industrial Average shed 0.2%, and the Nasdaq Composite fell nearly 1%.  Tech stocks were dragging on the Nasdaq earlier in the day as Wall Street took some profit on AI stocks and bought up a few laggards, including value, small-caps, and dividend stocks.',
   'pubDate': '2025-09-23T21:51:00Z',
   'displayTime': '2025-09-23T21:51:00Z',
   'isHosted': False,
   'bypassModal': False,
   'previewUrl': 'https://finance.yahoo.com/m/cd779a9a-0ebe-3d30-a502-6040f8474753/review-preview-the-rally.html',
   'thumbnail': {'originalUrl': 'https://media.zenfs.com/en/Barrons.com/12ea0f43d8f8307249a68d1cb2e0587e',
    'origi

In [2]:
import requests
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=9R341OB6CJV45U30'
r = requests.get(url)
data = r.json()
print(data)

{'items': '50', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': 'Apple App Store Revenue Jumps 10%, Analyst Sees AI, App Diversification Driving Long-Term Growth - Alphabet  ( NASDAQ:GOOG ) , Apple  ( NASDAQ:AAPL ) ', 'url': 'https://www.benzinga.com/analyst-stock-ratings/analyst-color/25/10/48019634/apple-app-store-revenue-jumps-10-analyst-sees-ai-app-diversification-driving-', 'time_published': '20251003T151743', 'authors': ['Anusuya Lahiri'], 'summary': "Apple's ( NASDAQ:AAPL ) App Store delivered strong results in the September quarter, with revenue climbing 10% year-over-year to $8.7 billion and downloads rising 2.5% to 9 billion. AAPL stock is up today. Get the details here.", 'banner_image': 'https://cdn.benzinga.com/files/images/story/2025/10/

In [4]:
from io import StringIO
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time  # For rate limiting

def get_stock_fii_dii_retail_df(stock_symbol):
    """
    Fetches FII, DII, and Retail (Public) holdings as a DataFrame from Screener.in.
    """
    url = f"https://www.screener.in/company/{stock_symbol.upper()}/"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        holdings_section = soup.find('section', id='shareholding')
        if holdings_section:
            html_string = str(holdings_section)
            data = StringIO(html_string)
            tables = pd.read_html(data)
            if tables and len(tables) > 0:
                df = tables[0]
                # Filter for FII, DII, and Public (Retail)
                mask = df['Unnamed: 0'].str.contains('FII|DII|Public', case=False, na=False)
                filtered_df = df[mask].copy()
                filtered_df = filtered_df.rename(columns={'Unnamed: 0': 'Category'})
                filtered_df['Category'] = filtered_df['Category'].str.replace(r'\s*\+\s*', '', regex=True).str.strip()
                percent_cols = [col for col in filtered_df.columns if col != 'Category']
                for col in percent_cols:
                    filtered_df[col] = filtered_df[col].astype(str).str.replace('%', '').astype(float)
                return filtered_df
            else:
                print(f"No holdings table found for {stock_symbol}.")
                return None
        else:
            print(f"Holdings section not found for {stock_symbol}.")
            return None
    except requests.RequestException as e:
        print(f"Request error for {stock_symbol}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error for {stock_symbol}: {e}")
        return None

def get_fii_dii_retail_multiple(tickers):
    """
    Fetches and combines FII/DII/Retail holdings for a list of tickers into one DataFrame.
    """
    all_data = []
    for ticker in tickers:
        df = get_stock_fii_dii_retail_df(ticker)
        if df is not None:
            df['Ticker'] = ticker.upper()
            all_data.append(df)
        time.sleep(2)  # Rate limit: Adjust as needed
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df
    else:
        print("No data found for any ticker.")
        return None

def print_fii_dii_retail_multiple_q(tickers):
    """
    Prints quarter-by-quarter holdings for each ticker in the list.
    """
    combined_df = get_fii_dii_retail_multiple(tickers)
    if combined_df is None or combined_df.empty:
        print("No data found.")
        return

    quarters = [col for col in combined_df.columns if col not in ['Category', 'Ticker']]
    for ticker in tickers:
        print(f"Quarter-wise FII, DII, and Retail Holdings for {ticker.upper()} (%):\n")
        ticker_df = combined_df[combined_df['Ticker'] == ticker.upper()]
        for quarter in quarters:
            print(f"{quarter}:")
            for idx, row in ticker_df.iterrows():
                print(f"  {row['Category']}: {row[quarter]:.2f} %")
            print()
        print("\n" + "="*50 + "\n")  # Separator between tickers

# Example usage: Process a list of tickers
ticker_list = ['RELIANCE', 'TCS', 'INFY']  # Add more, e.g., Nifty 200 symbols
combined_df = get_fii_dii_retail_multiple(ticker_list)

if combined_df is not None:
    # print("Combined DataFrame:")
    # print(combined_df.round(2))  # Rounded for readability
    # Optional: Export to CSV
    combined_df.to_csv('holdings_multi_tickers.csv', index=False)
    # print("\nQuarter-by-Quarter Print:")
    # print_fii_dii_retail_multiple_q(ticker_list)


In [14]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Intraday Pivot Strategy (Bounce + Breakout) — NSE (Yahoo Finance)
-----------------------------------------------------------------
What’s new in this build:
- Robust pivots from DAILY previous day.
- Auto-resolve session date to last tradable day <= TARGET_DATE (IST).
- Entries: wick-touch bounce and crossover breakout (configurable).
- Exits: TP/SL/EOD with **min_hold_bars** to avoid same-bar exits.
- Metrics printed to console and saved in summary:
    n_trades, wins, win%, avg_pnl, avg_win, avg_loss, expectancy, median_pnl,
    avg_hold_min, gross_pnl, max_gain, max_loss.
- EMA filter crash fixed: comparisons use scalars only.

Outputs:
  outputs/INTRADAY_<YYYY-MM-DD>/summary.csv
  outputs/INTRADAY_<YYYY-MM-DD>/signals_all.csv
  outputs/INTRADAY_<YYYY-MM-DD>/<SYMBOL>_trades.csv
"""

import os, math, logging, warnings
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from datetime import timedelta

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=FutureWarning)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("pivot_intraday_metrics")

try:
    import yfinance as yf
except Exception as e:
    raise SystemExit("Please install yfinance: pip install yfinance") from e

# =========================
# CONFIG
# =========================
SYMBOLS: List[str] = ['ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DRREDDY.NS', 'DUMMYTATAM.NS', 'EICHERMOT.NS', 'ETERNAL.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ITC.NS', 'INFY.NS', 'INDIGO.NS', 'JSWSTEEL.NS', 'JIOFIN.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'NTPC.NS', 'NESTLEIND.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SHRIRAMFIN.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'TRENT.NS', 'ULTRACEMCO.NS', 'WIPRO.NS']


# None = run for "today" (IST). Or set like "2025-10-16"
TARGET_DATE: Optional[str] = "2025-10-16" #None

@dataclass
class Config:
    interval: str = "5m"
    session_t_start: str = "09:30"
    session_t_end: str   = "15:00"

    # Filters (start OFF to confirm signals work; turn on after)
    use_ema_filter: bool = True
    ema_len: int = 9
    use_rsi_filter: bool = False
    rsi_len: int = 14
    rsi_long_min: float = 30.0
    rsi_long_max: float = 60.0
    rsi_short_min: float = 40.0
    rsi_short_max: float = 70.0

    # Strategies
    enable_bounce: bool = True
    enable_breakout: bool = True

    # Entry confirmations / tolerances
    touch_tolerance_bps: float = 20.0
    breakout_close_confirm: bool = True
    breakout_body_bps: float = 3.0
    bounce_use_wick_touch: bool = True
    breakout_use_crossover: bool = True

    # Risk: SL/TP
    sl_buffer_bps: float = 8.0

    # Execution model
    min_hold_bars: int = 1          # <- prevents same-bar TP/SL; set 0 for immediate
    allow_intrabar_tp_sl: bool = True  # still checks H/L but only after min_hold_bars

cfg = Config()
IST = "Asia/Kolkata"

# =========================
# HELPERS
# =========================
def ensure_ist_index(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    if df.index.tz is None:
        df = df.tz_localize("UTC").tz_convert(IST)
    else:
        df = df.tz_convert(IST)
    return df

def ema(series: pd.Series, period: int) -> pd.Series:
    return series.ewm(span=period, adjust=False).mean()

def rsi(series: pd.Series, length: int) -> pd.Series:
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    roll_up = up.ewm(alpha=1/length, adjust=False).mean()
    roll_dn = down.ewm(alpha=1/length, adjust=False).mean()
    rs = roll_up / roll_dn.replace(0, np.nan)
    return 100 - (100 / (1 + rs))

def in_bps(a: float, b: float) -> float:
    if a == 0 or b == 0:
        return np.inf
    return abs(a - b) / b * 1e4

def safe_float(x) -> float:
    try:
        return float(x)
    except Exception:
        return np.nan

def compute_pivots(h: float, l: float, c: float) -> Dict[str, float]:
    pp = (h + l + c) / 3.0
    r1 = 2*pp - l
    s1 = 2*pp - h
    r2 = pp + (h - l)
    s2 = pp - (h - l)
    r3 = h + 2*(pp - l)
    s3 = l - 2*(h - pp)
    return {"PP": pp, "R1": r1, "S1": s1, "R2": r2, "S2": s2, "R3": r3, "S3": s3}

def session_slice(df: pd.DataFrame, session_date: pd.Timestamp) -> pd.DataFrame:
    start = pd.Timestamp(f"{session_date.date()} {cfg.session_t_start}", tz=IST)
    end   = pd.Timestamp(f"{session_date.date()} {cfg.session_t_end}", tz=IST)
    return df.loc[(df.index >= start) & (df.index <= end)].copy()

def resolve_trading_session(df_intraday: pd.DataFrame, target_date: Optional[pd.Timestamp]) -> Optional[pd.Timestamp]:
    """Return the last session date in intraday index <= target_date (IST)."""
    if df_intraday.empty:
        return None
    idx = df_intraday.index.tz_convert(IST)
    dates = pd.DatetimeIndex(sorted({pd.Timestamp(d.date(), tz=IST) for d in idx}))
    if target_date is None:
        return dates[-1]
    mask = dates <= pd.Timestamp(target_date.date(), tz=IST)
    if not mask.any():
        return None
    return dates[mask][-1]

def get_prev_daily_hlc(sym: str, session_date: pd.Timestamp) -> Tuple[float, float, float]:
    start = (session_date - timedelta(days=30)).strftime("%Y-%m-%d")
    end   = (session_date + timedelta(days=1)).strftime("%Y-%m-%d")
    ddf = yf.download(sym, start=start, end=end, interval="1d", auto_adjust=False, progress=False)
    if ddf.empty:
        raise ValueError("daily_data_empty")
    ddf = ensure_ist_index(ddf)
    ddf["d"] = ddf.index.tz_convert(IST).date
    target = pd.Timestamp(session_date.date(), tz=IST).date()
    avail = ddf[ddf["d"] <= target]
    if avail.empty:
        raise ValueError("no_daily_on_or_before_session")
    prev = ddf[ddf["d"] < target]
    row = (prev.iloc[-1] if not prev.empty else avail.iloc[-1])
    h, l, c = safe_float(row["High"]), safe_float(row["Low"]), safe_float(row["Close"])
    return h, l, c

def choose_tp_sl(level_key_entry: str, piv: Dict[str, float], side: str) -> Tuple[float, float]:
    bps = cfg.sl_buffer_bps
    if side == "LONG":
        if level_key_entry == "S1":     # bounce long
            tp = piv["PP"]; sl = piv["S1"] * (1 - bps/1e4)
        elif level_key_entry == "R1":   # breakout long
            tp = piv["R2"]; sl = piv["R1"] * (1 - bps/1e4)
        else:
            tp = piv["R1"]; sl = piv["PP"] * (1 - bps/1e4)
    else:
        if level_key_entry == "R1":     # bounce short
            tp = piv["PP"]; sl = piv["R1"] * (1 + bps/1e4)
        elif level_key_entry == "S1":   # breakout short
            tp = piv["S2"]; sl = piv["S1"] * (1 + bps/1e4)
        else:
            tp = piv["S1"]; sl = piv["PP"] * (1 + bps/1e4)
    return tp, sl

def passes_filters(close_val: float, ema_val: float, rsi_val: float, side: str) -> bool:
    """All inputs must be scalars. Prevents 'truth value of a Series is ambiguous' errors."""
    c = safe_float(close_val)
    e = safe_float(ema_val)
    r = safe_float(rsi_val)

    if cfg.use_ema_filter and np.isfinite(e):
        if side == "LONG" and not (c > e):  return False
        if side == "SHORT" and not (c < e): return False

    if cfg.use_rsi_filter and np.isfinite(r):
        if side == "LONG"  and not (cfg.rsi_long_min  <= r <= cfg.rsi_long_max):   return False
        if side == "SHORT" and not (cfg.rsi_short_min <= r <= cfg.rsi_short_max): return False

    return True

# =========================
# CORE
# =========================
def run_symbol(sym: str, target_date: Optional[pd.Timestamp]) -> Tuple[pd.DataFrame, Dict]:
    # Pull intraday window ~14d
    start = (pd.Timestamp.now(tz=IST) - timedelta(days=14)).strftime("%Y-%m-%d")
    end   = (pd.Timestamp.now(tz=IST) + timedelta(days=1)).strftime("%Y-%m-%d")
    idf = yf.download(sym, start=start, end=end, interval=cfg.interval, auto_adjust=True, progress=False, prepost=False, threads=True)
    idf = ensure_ist_index(idf)
    if idf.empty:
        log.warning(f"{sym}: intraday empty; skip.")
        return pd.DataFrame(), {"symbol": sym, "error": "no_intraday"}

    # Resolve session
    session_date = resolve_trading_session(idf, target_date)
    if session_date is None:
        log.warning(f"{sym}: cannot resolve trading session for {target_date}; skip.")
        return pd.DataFrame(), {"symbol": sym, "error": "no_session"}

    sess = session_slice(idf, session_date)
    if sess.empty:
        log.warning(f"{sym}: no bars in session {session_date.date()}; skip.")
        return pd.DataFrame(), {"symbol": sym, "error": "empty_session"}

    # Technicals
    idf["EMA"] = ema(idf["Close"], cfg.ema_len)
    idf["RSI"] = rsi(idf["Close"], cfg.rsi_len)
    sess["EMA"] = idf.loc[sess.index, "EMA"]
    sess["RSI"] = idf.loc[sess.index, "RSI"]

    # Pivots from previous DAILY
    try:
        ph, pl, pc = get_prev_daily_hlc(sym, session_date)
    except Exception as e:
        log.warning(f"{sym}: pivot daily fetch failed: {e}")
        return pd.DataFrame(), {"symbol": sym, "error": "pivot_fail"}
    piv = compute_pivots(ph, pl, pc)

    log.info(f"{sym} | Session={session_date.date()} | PrevDaily H/L/C=({ph:.2f},{pl:.2f},{pc:.2f}) | PP={piv['PP']:.2f} R1={piv['R1']:.2f} S1={piv['S1']:.2f}")

    # Sim loop
    in_pos = False
    side = None
    entry_px = np.nan
    entry_ts = None
    tp_px = sl_px = np.nan
    entry_reason = ""
    bars_since_entry = 0

    recs = []
    body_bps = cfg.breakout_body_bps
    prev_close = None

    for ts, row in sess.iterrows():
        c = safe_float(row["Close"]); o = safe_float(row["Open"]); h = safe_float(row["High"]); l = safe_float(row["Low"])
        ev = safe_float(row.get("EMA", np.nan))
        rv = safe_float(row.get("RSI", np.nan))

        # Exits first (only after min_hold_bars)
        if in_pos and bars_since_entry >= cfg.min_hold_bars:
            hit_tp = (side == "LONG" and h >= tp_px) or (side == "SHORT" and l <= tp_px)
            hit_sl = (side == "LONG" and l <= sl_px) or (side == "SHORT" and h >= sl_px)

            if hit_tp and hit_sl:
                # Conservative: assume SL first
                hit_tp = False

            if (cfg.allow_intrabar_tp_sl and (hit_tp or hit_sl)):
                exit_px = tp_px if hit_tp else sl_px
                reason = "TP" if hit_tp else "SL"
                pnl = (exit_px - entry_px) * (1 if side == "LONG" else -1)
                recs.append({
                    "symbol": sym, "entry_time": entry_ts, "entry_price": entry_px,
                    "side": side, "reason_entry": entry_reason,
                    "exit_time": ts, "exit_price": exit_px,
                    "reason_exit": reason, "pnl": pnl, "ret_pct": pnl/entry_px*100.0,
                    "hold_min": (ts - entry_ts).total_seconds() / 60.0,
                })
                in_pos = False
                side = None
                entry_px = np.nan
                entry_ts = None
                tp_px = sl_px = np.nan
                entry_reason = ""
                bars_since_entry = 0
                prev_close = c
                continue

        # Entries (flat)
        if not in_pos:
            # ---- BOUNCE ----
            if cfg.enable_bounce:
                # Long bounce @ S1: wick touches S1 and bullish close back above S1
                if cfg.bounce_use_wick_touch:
                    touched_s1 = (l <= piv["S1"] <= h)
                    bullish = (c > o) and (c > piv["S1"])
                    if touched_s1 and bullish and passes_filters(c, ev, rv, "LONG"):
                        in_pos = True; side = "LONG"; entry_px = c; entry_ts = ts
                        tp_px, sl_px = choose_tp_sl("S1", piv, "LONG")
                        entry_reason = "Bounce@S1 (wick touch + bullish close)"
                        bars_since_entry = 0

                else:
                    if in_bps(c, piv["S1"]) <= cfg.touch_tolerance_bps and c > piv["S1"] and c > o and passes_filters(c, ev, rv, "LONG"):
                        in_pos = True; side = "LONG"; entry_px = c; entry_ts = ts
                        tp_px, sl_px = choose_tp_sl("S1", piv, "LONG")
                        entry_reason = "Bounce@S1 (proximity + bullish close)"
                        bars_since_entry = 0

                # Short bounce @ R1: wick touches R1 and bearish close back below
                if (not in_pos) and cfg.bounce_use_wick_touch:
                    touched_r1 = (l <= piv["R1"] <= h)
                    bearish = (o > c) and (c < piv["R1"])
                    if touched_r1 and bearish and passes_filters(c, ev, rv, "SHORT"):
                        in_pos = True; side = "SHORT"; entry_px = c; entry_ts = ts
                        tp_px, sl_px = choose_tp_sl("R1", piv, "SHORT")
                        entry_reason = "Bounce@R1 (wick touch + bearish close)"
                        bars_since_entry = 0

                elif (not in_pos):
                    if in_bps(c, piv["R1"]) <= cfg.touch_tolerance_bps and c < piv["R1"] and (o > c) and passes_filters(c, ev, rv, "SHORT"):
                        in_pos = True; side = "SHORT"; entry_px = c; entry_ts = ts
                        tp_px, sl_px = choose_tp_sl("R1", piv, "SHORT")
                        entry_reason = "Bounce@R1 (proximity + bearish close)"
                        bars_since_entry = 0

            # ---- BREAKOUT ----
            if (not in_pos) and cfg.enable_breakout:
                # Long above R1
                body_ok = ((c - o)/max(o, 1e-9)*1e4) >= body_bps if cfg.breakout_close_confirm else True
                cross_ok = True
                if cfg.breakout_use_crossover and prev_close is not None:
                    cross_ok = (prev_close <= piv["R1"] and c > piv["R1"]) or (h >= piv["R1"] and o <= piv["R1"])
                if (c > piv["R1"]) and body_ok and cross_ok and passes_filters(c, ev, rv, "LONG"):
                    in_pos = True; side = "LONG"; entry_px = c; entry_ts = ts
                    tp_px, sl_px = choose_tp_sl("R1", piv, "LONG")
                    entry_reason = "Breakout>R1 (cross + momentum)"
                    bars_since_entry = 0

                # Short below S1
                if not in_pos:
                    body_ok_s = ((o - c)/max(o, 1e-9)*1e4) >= body_bps if cfg.breakout_close_confirm else True
                    cross_ok_s = True
                    if cfg.breakout_use_crossover and prev_close is not None:
                        cross_ok_s = (prev_close >= piv["S1"] and c < piv["S1"]) or (l <= piv["S1"] and o >= piv["S1"])
                    if (c < piv["S1"]) and body_ok_s and cross_ok_s and passes_filters(c, ev, rv, "SHORT"):
                        in_pos = True; side = "SHORT"; entry_px = c; entry_ts = ts
                        tp_px, sl_px = choose_tp_sl("S1", piv, "SHORT")
                        entry_reason = "Breakdown<S1 (cross + momentum)"
                        bars_since_entry = 0

        # increment bar counter if in position
        if in_pos:
            bars_since_entry += 1

        prev_close = c

    # EOD flatten
    if in_pos:
        exit_ts = sess.index[-1]
        exit_px = safe_float(sess["Close"].iloc[-1])     # <- avoid KeyError: -1
        pnl = (exit_px - entry_px) * (1 if side == "LONG" else -1)
        recs.append({
            "symbol": sym, "entry_time": entry_ts, "entry_price": entry_px,
            "side": side, "reason_entry": entry_reason,
            "exit_time": exit_ts, "exit_price": exit_px,
            "reason_exit": "EOD", "pnl": pnl, "ret_pct": pnl/entry_px*100.0,
            "hold_min": (exit_ts - entry_ts).total_seconds() / 60.0,
        })

    trades = pd.DataFrame(recs)

    # === Per-symbol metrics ===
    metrics = {
        "n_trades": int(len(trades)),
        "wins": int((trades["pnl"] > 0).sum()) if not trades.empty else 0,
        "win_rate_pct": float((trades["pnl"] > 0).mean()*100) if not trades.empty else 0.0,
        "avg_pnl": float(trades["pnl"].mean()) if not trades.empty else 0.0,
        "avg_win": float(trades.loc[trades["pnl"] > 0, "pnl"].mean()) if not trades.empty and (trades["pnl"] > 0).any() else 0.0,
        "avg_loss": float(trades.loc[trades["pnl"] <= 0, "pnl"].mean()) if not trades.empty and (trades["pnl"] <= 0).any() else 0.0,
        "expectancy": float(trades["pnl"].mean()) if not trades.empty else 0.0,  # per-trade average pnl
        "median_pnl": float(trades["pnl"].median()) if not trades.empty else 0.0,
        "avg_hold_min": float(trades["hold_min"].mean()) if not trades.empty else 0.0,
        "gross_pnl": float(trades["pnl"].sum()) if not trades.empty else 0.0,
        "max_gain": float(trades["pnl"].max()) if not trades.empty else 0.0,
        "max_loss": float(trades["pnl"].min()) if not trades.empty else 0.0,
    }

    log.info(
        f"{sym} | Bars={len(sess)} | Trades={metrics['n_trades']} | "
        f"Win%={metrics['win_rate_pct']:.1f} | AvgPnL={metrics['avg_pnl']:.4f} | "
        f"AvgWin={metrics['avg_win']:.4f} | AvgLoss={metrics['avg_loss']:.4f} | "
        f"Expectancy={metrics['expectancy']:.4f} | AvgHold={metrics['avg_hold_min']:.1f}m"
    )

    meta = {
        "symbol": sym,
        "session_date": str(session_date.date()),
        "pivots": piv,
        "prev_daily": {"H": ph, "L": pl, "C": pc},
        "bars": len(sess),
        **metrics,
    }
    return trades, meta


def main():
    # Resolve target date (IST)
    if TARGET_DATE:
        target = pd.Timestamp(TARGET_DATE, tz=IST)
    else:
        target = pd.Timestamp.now(tz=IST)
    target = pd.Timestamp(target.date(), tz=IST)

    out_dir = f"outputs/INTRADAY_{target.date()}"
    os.makedirs(out_dir, exist_ok=True)

    all_entries = []
    summaries = []

    for sym in SYMBOLS:
        log.info(f"=== {sym} : running ===")
        trades_df, meta = run_symbol(sym, target)
        if not trades_df.empty:
            fpath = os.path.join(out_dir, f"{sym.replace('^','IDX_').replace('.','_')}_trades.csv")
            trades_df.to_csv(fpath, index=False)

            for _, r in trades_df.iterrows():
                all_entries.append({
                    "symbol": sym, "time": r["entry_time"], "side": r["side"],
                    "entry": r["entry_price"], "reason": r["reason_entry"]
                })

        # Append summary row regardless (so you can see pivots, bars, metrics even if 0 trades)
        summaries.append({
            "symbol": sym,
            "session_date": meta.get("session_date"),
            "bars": meta.get("bars"),
            "PP": meta.get("pivots", {}).get("PP", np.nan),
            "R1": meta.get("pivots", {}).get("R1", np.nan),
            "S1": meta.get("pivots", {}).get("S1", np.nan),
            "n_trades": meta.get("n_trades", 0),
            "wins": meta.get("wins", 0),
            "win_rate_pct": round(float(meta.get("win_rate_pct", 0.0)), 2),
            "avg_pnl": round(float(meta.get("avg_pnl", 0.0)), 6),
            "avg_win": round(float(meta.get("avg_win", 0.0)), 6),
            "avg_loss": round(float(meta.get("avg_loss", 0.0)), 6),
            "expectancy": round(float(meta.get("expectancy", 0.0)), 6),
            "median_pnl": round(float(meta.get("median_pnl", 0.0)), 6),
            "avg_hold_min": round(float(meta.get("avg_hold_min", 0.0)), 2),
            "gross_pnl": round(float(meta.get("gross_pnl", 0.0)), 6),
            "max_gain": round(float(meta.get("max_gain", 0.0)), 6),
            "max_loss": round(float(meta.get("max_loss", 0.0)), 6),
        })

    # Save consolidated
    pd.DataFrame(all_entries).to_csv(os.path.join(out_dir, "signals_all.csv"), index=False)
    pd.DataFrame(summaries).to_csv(os.path.join(out_dir, "summary.csv"), index=False)
    log.info(f"Saved -> {out_dir}/signals_all.csv and summary.csv; per-symbol trades saved too. Done.")

if __name__ == "__main__":
    main()


2025-10-19 13:15:36 | INFO | === ADANIENT.NS : running ===
2025-10-19 13:15:36 | INFO | ADANIENT.NS | Session=2025-10-16 | PrevDaily H/L/C=(2561.40,2527.30,2532.80) | PP=2540.50 R1=2553.70 S1=2519.60
2025-10-19 13:15:36 | INFO | ADANIENT.NS | Bars=67 | Trades=0 | Win%=0.0 | AvgPnL=0.0000 | AvgWin=0.0000 | AvgLoss=0.0000 | Expectancy=0.0000 | AvgHold=0.0m
2025-10-19 13:15:36 | INFO | === ADANIPORTS.NS : running ===
2025-10-19 13:15:37 | INFO | ADANIPORTS.NS | Session=2025-10-16 | PrevDaily H/L/C=(1456.30,1430.00,1450.70) | PP=1445.67 R1=1461.33 S1=1435.03
2025-10-19 13:15:37 | INFO | ADANIPORTS.NS | Bars=67 | Trades=0 | Win%=0.0 | AvgPnL=0.0000 | AvgWin=0.0000 | AvgLoss=0.0000 | Expectancy=0.0000 | AvgHold=0.0m
2025-10-19 13:15:37 | INFO | === APOLLOHOSP.NS : running ===
2025-10-19 13:15:38 | INFO | APOLLOHOSP.NS | Session=2025-10-16 | PrevDaily H/L/C=(7850.00,7740.50,7826.00) | PP=7805.50 R1=7870.50 S1=7761.00
2025-10-19 13:15:38 | INFO | APOLLOHOSP.NS | Bars=67 | Trades=0 | Win%=0.0 |

In [10]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Donchian(40) + 3-bar Pullback + Breakout — Daily Backtest
- Data: yfinance daily bars
- Entry: touch band -> >=3-bar pullback -> breakout of *touched* level
- Execute at next day's OPEN
- Risk: rupee SL/TP per trade (default ₹2k / ₹4k)
- Portfolio: fixed capital, equal allocation, max 5 concurrent positions
- Short side supported (mirror logic)
Outputs: trades.csv + console summary
"""

import os, logging
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import numpy as np
import pandas as pd

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    tickers: List[str] = None
    period: str = "20y"             # daily history window
    interval: str = "1d"            # DAILY timeframe
    donchian_len: int = 40
    pullback_bars: int = 3
    capital: float = 5_00_000.0     # total capital in ₹
    max_concurrent: int = 5
    sl_rupees: float = 2_000.0
    tp_rupees: float = 10000.0
    both_hit_policy: str = "sl_first"   # or "tp_first"
    out_dir: str = "outputs/donchian_pullback_1d"
    trades_csv: str = "trades.csv"
    seed: int = 42

CFG = Config(
    tickers=[
        "RELIANCE.NS","HDFCBANK.NS","TCS.NS","INFY.NS","ICICIBANK.NS",
        "LT.NS","SBIN.NS","AXISBANK.NS","ITC.NS","BHARTIARTL.NS"
    ]
)

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("donchian_pullback_1d")

# =========================
# HELPERS
# =========================
def ensure_dir(p: str) -> None:
    os.makedirs(p, exist_ok=True)

def yf_download(symbol: str, period: str, interval: str) -> pd.DataFrame:
    import yfinance as yf
    df = yf.download(symbol, period=period, interval=interval,
                     progress=False, threads=False, auto_adjust=False, multi_level_index=False)
    if df is None or df.empty:
        return pd.DataFrame()
    df = df.rename(columns=str.title)
    return df[["Open","High","Low","Close","Adj Close","Volume"]]

def donchian(df: pd.DataFrame, length: int) -> pd.DataFrame:
    hi = df["High"].rolling(length, min_periods=length).max()
    lo = df["Low"].rolling(length, min_periods=length).min()
    mid = (hi + lo) / 2.0
    out = df.copy()
    out["DC_U"] = hi
    out["DC_L"] = lo
    out["DC_M"] = mid
    return out

def find_entry_events(df: pd.DataFrame, pullback_bars: int, side: str) -> List[Tuple[pd.Timestamp, float]]:
    """
    Returns (trigger_day, touched_level). Fill is next day OPEN.
    """
    d = df
    events = []
    if side == "long":
        touch = d["High"] >= d["DC_U"]
        lvl_series = d["DC_U"]

        def pb_ok(i, j):
            # bars i+1..j-1 are pullback bars: High < level and Close < level
            return (j - i - 1) >= pullback_bars and \
                   (d.iloc[i+1:j][["High","Close"]] < lvl_series.iloc[i]).all().all()

        def trigger(j, level):
            return d["High"].iloc[j] >= level
    else:
        touch = d["Low"] <= d["DC_L"]
        lvl_series = d["DC_L"]

        def pb_ok(i, j):
            return (j - i - 1) >= pullback_bars and \
                   (d.iloc[i+1:j][["Low","Close"]] > lvl_series.iloc[i]).all().all()

        def trigger(j, level):
            return d["Low"].iloc[j] <= level

    idxs = np.where(touch.values)[0]
    for i in idxs:
        level = float(lvl_series.iloc[i])
        j_start = i + pullback_bars + 1
        if j_start >= len(d): 
            continue
        for j in range(j_start, len(d)):
            if trigger(j, level):
                events.append((d.index[j], level))
                break
    return events

# =========================
# SIMULATOR
# =========================
class Position:
    def __init__(self, sym: str, side: str, qty: int, entry_t: pd.Timestamp, entry_px: float,
                 tp_rupees: float, sl_rupees: float, both_hit_policy: str):
        self.sym = sym
        self.side = side
        self.qty = qty
        self.entry_t = entry_t
        self.entry_px = entry_px
        self.tp_rupees = tp_rupees
        self.sl_rupees = sl_rupees
        self.both_hit_policy = both_hit_policy

        self.tp_px = self._calc_tp()
        self.sl_px = self._calc_sl()

    def _calc_tp(self) -> float:
        per_share = self.tp_rupees / self.qty
        return self.entry_px + per_share if self.side == "long" else self.entry_px - per_share

    def _calc_sl(self) -> float:
        per_share = self.sl_rupees / self.qty
        return self.entry_px - per_share if self.side == "long" else self.entry_px + per_share

    def check_exit_on_bar(self, o: float, h: float, l: float, c: float) -> Optional[Tuple[float,str]]:
        if self.side == "long":
            hit_sl = l <= self.sl_px
            hit_tp = h >= self.tp_px
        else:
            hit_sl = h >= self.sl_px
            hit_tp = l <= self.tp_px

        if hit_sl and hit_tp:
            if self.both_hit_policy == "tp_first":
                return (self.tp_px, "TP+SL_same_bar_tp_first")
            return (self.sl_px, "SL+TP_same_bar_sl_first")
        if hit_sl: return (self.sl_px, "SL")
        if hit_tp: return (self.tp_px, "TP")
        return None

    def pnl(self, exit_px: float) -> float:
        return (exit_px - self.entry_px) * self.qty if self.side == "long" else (self.entry_px - exit_px) * self.qty

def run_backtest() -> pd.DataFrame:
    ensure_dir(CFG.out_dir)
    rng = np.random.default_rng(CFG.seed)

    # Data
    data: Dict[str, pd.DataFrame] = {}
    for s in CFG.tickers:
        df = yf_download(s, CFG.period, CFG.interval)
        if df.empty:
            log.warning(f"[{s}] no data, skipped.")
            continue
        df = donchian(df, CFG.donchian_len).dropna()
        data[s] = df
        log.info(f"[{s}] daily bars={len(df)}")

    if not data:
        raise SystemExit("No data available.")

    # Entry events (trigger on day T; fill at T+1 open)
    events = []  # (trigger_day, sym, side, level)
    for s, df in data.items():
        for t, lvl in find_entry_events(df, CFG.pullback_bars, "long"):
            events.append((t, s, "long", lvl))
        for t, lvl in find_entry_events(df, CFG.pullback_bars, "short"):
            events.append((t, s, "short", lvl))

    if not events:
        log.warning("No entry events found.")
        return pd.DataFrame(columns=["symbol","side","entry_time","entry_price","qty","exit_time","exit_price","pnl","exit_reason"])

    # Sort by date; random tie-break to avoid symbol bias on same day
    events.sort(key=lambda x: (x[0], rng.random()))

    # Global clock: union of all dates
    all_dates = sorted(set(pd.concat([df.index.to_series() for df in data.values()]).index))

    # Pre-cache OHLC
    ohlc: Dict[str, pd.DataFrame] = {s: df[["Open","High","Low","Close"]] for s, df in data.items()}

    # Events by day
    from collections import defaultdict
    ev_by_day = defaultdict(list)
    for t, s, side, lvl in events:
        ev_by_day[t].append((s, side, lvl))

    allocation_per_pos = CFG.capital / CFG.max_concurrent
    open_pos: Dict[str, Position] = {}
    trades = []

    def close_pos(sym: str, pos: Position, d: pd.Timestamp, px: float, reason: str):
        trades.append({
            "symbol": sym,
            "side": pos.side,
            "entry_time": pos.entry_t,
            "entry_price": pos.entry_px,
            "qty": pos.qty,
            "exit_time": d,
            "exit_price": px,
            "pnl": pos.pnl(px),
            "exit_reason": reason
        })
        open_pos.pop(sym, None)

    # Iterate by trading days
    for i, d in enumerate(all_dates):
        # Exits first
        for sym, pos in list(open_pos.items()):
            df = ohlc[sym]
            if d not in df.index:  # symbol holiday
                continue
            row = df.loc[d]
            hit = pos.check_exit_on_bar(row["Open"], row["High"], row["Low"], row["Close"])
            if hit:
                px, reason = hit
                close_pos(sym, pos, d, float(px), reason)

        # Entries from triggers on day d (fill on day d+1 open)
        if d in ev_by_day and (i + 1) < len(all_dates):
            next_d = all_dates[i+1]
            for (sym, side, lvl) in ev_by_day[d]:
                if len(open_pos) >= CFG.max_concurrent:
                    break
                if sym in open_pos:
                    continue
                df = ohlc[sym]
                if next_d not in df.index:
                    continue
                px = float(df.loc[next_d]["Open"])
                qty = int(allocation_per_pos // px)
                if qty <= 0:
                    continue
                pos = Position(sym, side, qty, next_d, px, CFG.tp_rupees, CFG.sl_rupees, CFG.both_hit_policy)
                open_pos[sym] = pos

    # Wrap up (leave open trades as-is; not force-closed)
    if not trades:
        log.info("No trades executed.")
        return pd.DataFrame(columns=["symbol","side","entry_time","entry_price","qty","exit_time","exit_price","pnl","exit_reason"])

    trades_df = pd.DataFrame(trades).sort_values("entry_time").reset_index(drop=True)
    return trades_df

def summarize(tr: pd.DataFrame) -> None:
    if tr.empty:
        print("No trades.")
        return
    total = tr["pnl"].sum()
    wins = (tr["pnl"] > 0).sum()
    winrate = 100.0 * wins / len(tr)
    avg = tr["pnl"].mean()
    eq = tr["pnl"].cumsum()
    dd = (eq - eq.cummax()).min()
    print(f"Trades={len(tr)} | Winrate={winrate:.1f}% | Net P&L=₹{total:,.2f} | Avg=₹{avg:,.2f} | MaxDD≈₹{dd:,.0f}")

def main():
    trades = run_backtest()
    ensure_dir(CFG.out_dir)
    path = os.path.join(CFG.out_dir, CFG.trades_csv)
    trades.to_csv(path, index=False)
    summarize(trades)
    print(f"\nCSV saved to: {path}")

if __name__ == "__main__":
    main()


2025-11-11 13:00:09 | INFO | [RELIANCE.NS] daily bars=4895
2025-11-11 13:00:09 | INFO | [HDFCBANK.NS] daily bars=4895
2025-11-11 13:00:10 | INFO | [TCS.NS] daily bars=4895
2025-11-11 13:00:10 | INFO | [INFY.NS] daily bars=4895
2025-11-11 13:00:10 | INFO | [ICICIBANK.NS] daily bars=4895
2025-11-11 13:00:11 | INFO | [LT.NS] daily bars=4895
2025-11-11 13:00:11 | INFO | [SBIN.NS] daily bars=4895
2025-11-11 13:00:12 | INFO | [AXISBANK.NS] daily bars=4895
2025-11-11 13:00:13 | INFO | [ITC.NS] daily bars=4895
2025-11-11 13:00:13 | INFO | [BHARTIARTL.NS] daily bars=4895


Trades=2332 | Winrate=16.6% | Net P&L=₹-8,000.00 | Avg=₹-3.43 | MaxDD≈₹-380,000

CSV saved to: outputs/donchian_pullback_1d/trades.csv
